In [23]:
# Imports
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
from skimage import color, io
from skimage.io import imsave

In [30]:
import sys

FLAGS = tf.app.flags.FLAGS
model_version = '0.0.1'

export_path_base = sys.argv[-1]
export_path = os.path.join(
      tf.compat.as_bytes(export_path_base),
      tf.compat.as_bytes(model_version))
print export_path

/root/.local/share/jupyter/runtime/kernel-afcf7348-7111-4ffa-a370-3cf3c56953fd.json/0.0.1


In [13]:
def convertToLab(image):
    lab = color.rgb2lab(image)
    X_batch = lab[:,:,0]
    Y_batch = lab[:,:,1:] / 128
    return X_batch.reshape(X_batch.shape+(1,)), Y_batch

def parseImage(filename):
    image = tf.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [400, 400], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

In [18]:
# file_test_paths = ['woman.jpg']
file_test_paths = map((lambda x: "../train4/" + x), os.listdir("../train4/"))

dataset = tf.data.Dataset.from_tensor_slices(file_test_paths)
dataset = dataset.map(parseImage)
dataset = dataset.map(lambda image:
    tuple(tf.py_func(
        convertToLab, [image], [tf.double, tf.double]
    ))
)
dataset = dataset.batch(1)

iterator = dataset.make_one_shot_iterator()

In [19]:
def conv2DRelu(X, W, B, strides, padding):
    # strides: [batch_step, height_step, width_step, channel_step]
    return tf.nn.relu(tf.nn.conv2d(X, W, strides=strides, padding=padding) + B)

def conv2DTanh(X, W, B, strides, padding):
    # strides: [batch_step, height_step, width_step, channel_step]
    return tf.nn.tanh(tf.nn.conv2d(X, W, strides=strides, padding=padding) + B)

def weight(width, height, input_channels, output_channels):
    # [width, height, input channel, output channel]
    return tf.Variable(tf.truncated_normal([width, height, input_channels, output_channels], stddev=0.1))

def bias(outputChannels):
    return tf.Variable(tf.zeros([outputChannels])) # bias for each output channel.

def upSampling2D(X, height, width):
    return tf.image.resize_images(X, [height, width], tf.image.ResizeMethod.NEAREST_NEIGHBOR)

def upSampleToOriginalSize(X, size):
    return tf.image.resize_images(X, size, tf.image.ResizeMethod.NEAREST_NEIGHBOR)

def Conv2D(X, input_channels, output_channels, scan=3, activation='relu', padding='SAME', strides=1):
    W = weight(scan, scan, input_channels, output_channels)
    B = bias(output_channels)
    if activation == 'relu':
        return conv2DRelu(X, W, B, [1,strides,strides,1], padding)
    else:
        return conv2DTanh(X, W, B, [1,strides,strides,1], padding)

In [20]:
Y_ = tf.placeholder(tf.float32, shape=[None, 400, 400, 2]) # True Value
X = tf.placeholder(tf.float32, shape=[None, 400, 400, 1]) # Input

Y1 = Conv2D(X, 1, 8, 3, 'relu', 'SAME', 2)
Y2 = Conv2D(Y1, 8, 8, 3, 'relu', 'SAME', 1)
Y3 = Conv2D(Y2, 8, 16, 3, 'relu', 'SAME', 1)
Y4 = Conv2D(Y3, 16, 16, 3, 'relu', 'SAME', 2)
Y5 = Conv2D(Y4, 16, 32, 3, 'relu', 'SAME', 1)
Y6 = Conv2D(Y5, 32, 32, 3, 'relu', 'SAME', 2)
Y7 = upSampling2D(Y6, 100, 100)
Y8 = Conv2D(Y7, 32, 32, 3, 'relu', 'SAME', 1)
Y9 = upSampling2D(Y8, 200, 200)
Y10 = Conv2D(Y9, 32, 16, 3, 'relu', 'SAME', 1)
Y11 = upSampling2D(Y10, 400, 400)
Y12 = Conv2D(Y11, 16, 2, 3, 'tanh', 'SAME', 1)

# Define the loss function
loss = tf.reduce_mean(tf.squared_difference(Y12, Y_), 1)

# Define an optimizer
optimizer = tf.train.RMSPropOptimizer(0.001).minimize(loss)

In [21]:
init = tf.global_variables_initializer()
next_element = iterator.get_next()
saver = tf.train.Saver()

with tf.Session() as sess:
    # initialize the variables
    sess.run(init)

    # initialize the queue threads to start to shovel data
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    print("from the train set:")
    # images, labels = iterator.get_next()
    step = 0
    while True:

        try:
            elem = sess.run(next_element)
            print "Step:", step
            for i in range(1):
                # print("Round:", i)
                _, luss = sess.run([optimizer, loss], feed_dict={
                    X: elem[0], Y_: elem[1]
                })
                # print("Loss:", luss[0][0][0])
            step += 1
            
            if step == 2000:
                saver.save(sess, './model/' + 'model.ckpt', global_step=step + 1)
                print "End of training dataset."
                break
        except tf.errors.OutOfRangeError:
            saver.save(sess, './model/' + 'model.ckpt', global_step=step + 1)
            print "End of training dataset."
            break

    # stop our queue threads and properly close the session
    coord.request_stop()
    coord.join(threads)
    sess.close()

from the train set:
Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Step: 50
Step: 51
Step: 52
Step: 53
Step: 54
Step: 55
Step: 56
Step: 57
Step: 58
Step: 59
Step: 60
Step: 61
Step: 62
Step: 63
Step: 64
Step: 65
Step: 66
Step: 67
Step: 68
Step: 69
Step: 70
Step: 71
Step: 72
Step: 73
Step: 74
Step: 75
Step: 76
Step: 77
Step: 78
Step: 79
Step: 80
Step: 81
Step: 82
Step: 83
Step: 84
Step: 85
Step: 86
Step: 87
Step: 88
Step: 89
Step: 90
Step: 91
Step: 92
Step: 93
Step: 94
Step: 95
Step: 96
Step: 97
Step: 98
Step: 99
Step: 100
Step: 101
Step: 102
Step: 103
Step: 104
Step: 105
Step: 106
Step: 107
Step: 108


In [22]:
file_test_paths = ['woman.jpg']

testDataset = tf.data.Dataset.from_tensor_slices(file_test_paths)
testDataset = testDataset.map(parseImage)
testDataset = testDataset.map(lambda image:
    tuple(tf.py_func(
        convertToLab, [image], [tf.double, tf.double]
    ))
)
testDataset = testDataset.batch(1)

testIterator = testDataset.make_one_shot_iterator()


def plotImage(image):
    plt.imshow(image)
    plt.show()


next_element = testIterator.get_next()
with tf.Session() as session:
    elem = session.run(next_element)
    ckpt = tf.train.get_checkpoint_state('./model/')
    saver.restore(session, ckpt.model_checkpoint_path)
    feed_dict = {X: elem[0], Y_: elem[1]}
    _, ab = session.run([optimizer, Y12], feed_dict)

    # Colorize output
    ab = ab * 128

    cur = np.zeros((400, 400, 3))
    cur[:, :, 0] = elem[0][0][:, :, 0]
    cur[:, :, 1:] = ab[0]
    print("PRINTING")
    imsave("notebook_test.jpg", color.lab2rgb(cur))
    imsave("okkar_gray_version.png", color.rgb2gray(color.lab2rgb(cur)))

INFO:tensorflow:Restoring parameters from ./model/model.ckpt-394
PRINTING


/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
